### Introducción

Este cuaderno cubre el proceso de desarrollo de un modelo de NLP para predecir el sentimiento de los tweets relacionados con el cambio climático. Cubrirá cuatro fases principales:

- EDA (Análisis Exploratorio de Datos): donde se mostrarán y entenderán algunas métricas sobre los datos adquiridos para saber qué pasos seguir en las siguientes fases.
- Procesamiento: donde los datos de los tweets serán limpiados, preparados y adaptados a los formatos aceptado por el modelo.

### Import de las funciones y librerias pertinentes

In [ ]:
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from funciones_auxiliares.funciones_preprocessing import preprocess_text
from funciones_auxiliares.funciones_rebalancing import undersample
from funciones_auxiliares.funciones_data_loading import read_data_from_excel, format_sentiment_column

### Extracción del dataframe original

In [ ]:
df = read_data_from_excel('tweets/tweets_cam_clim_etiquetados.xlsx')
df = format_sentiment_column(df)

### Analisis exploratorio de los datos

Analizando la distribucion de las clases del dataset podemos ver que hay un gran desbalance entre sentimientos negativos (0), neutrales (1) y positivos (2). Considerando que el dataset utilizado no esta formado for varios miles de tweets, este desbalance puede afectar negativamente al modelo.

In [ ]:
df['Sentiment'].value_counts().plot(kind='bar')
# Añadimos titulo y ylabel
plt.title('Número de tweets por sentimiento')
plt.ylabel('Número de tweets')

Además, podemos ver como el numero de palabras utilizadas por los tweets de cada sentimiento varia considerablemente (diferencia de ∼25% entre el la media del numero de palabras usadas por tweets neutros y el conjunto de positivos y negativos). Pese a ello, la funcion de densidad de probabilidad de los tres subconjuntos es relativamente similar, por lo que, asumiendo que el descarte de las palabras menos relevantes homogeneizará aun mas los subconjuntos en numero de palabras, esto no tendría por qué ser un problema.

In [ ]:
# Distirbucion del numero de palabras por tweet por sentimiento
df['Number of words'] = df['RawContent'].apply(lambda x: len(x.split()))

# Plot boxplot of number of words per sentiment
df.boxplot(column='Number of words', by='Sentiment')



# Cuartiles del numero de palabras por tweet por sentimiento
df.groupby('Sentiment')['Number of words'].describe()

Los tweets son una fuente de informacion particularmente problemática, ya que no tienden a seguir una corrección ortográfica, sintáctica o gramatical, utilizan símbolos como emojis o puntuaciones para la representacion de emociones, etc. Uno de los pasos mas relevantes para su uso es el determinar que elementos se utilizan para poder establecer un preprocesamiento adecuado. En la siguiente tabla podemos ver los emojis utilizados:

In [ ]:
import emoji
emojis = set()
n_tweets = 0
n_emojis = 0
for tweet in df['RawContent']:
    emojis.update([c for c in tweet if c in emoji.unicode_codes.EMOJI_DATA])
    n_tweets += 1 if len([c for c in tweet if c in emoji.unicode_codes.EMOJI_DATA]) > 0 else 0
    n_emojis += len([c for c in tweet if c in emoji.unicode_codes.EMOJI_DATA])
    
print('Numero de emojis:', n_emojis)
print('Numero de emojis distintos:', len(emojis))
print('Numero de tweets con emojis:', n_tweets)

emojis = list(emojis)
emojis_count = [0]*len(emojis)
for tweet in df['RawContent']:
    for i, emoji in enumerate(emojis):
        if emoji in tweet:
            emojis_count[i] += 1

emojis_df = pd.DataFrame({'Emoji': emojis, 'Count': emojis_count})
emojis_df.sort_values('Count', ascending=False, inplace=True)
emojis_df

Es interesante conocer también qué palabras que no aparecen en el corpus de español se han utilizado para ver si deben ser o no consideradas en el proceso. Sin embargo, puede verse como las faltas ortográficas, el uso de simbolos de puntuacion al lado de las palabras, los hastags y otros simbolos comunes en los tweets hacen que el numero de palabras consideradas como no validas en el corpus sea enorme.

In [ ]:
import nltk
#nltk.download('wordnet')
#nltk.download('omw-1.4')
from nltk.corpus import wordnet

non_words = set()
for tweet in df['RawContent']:
    non_words.update([word for word in tweet.replace('.', '').replace(',', '').replace('?', '').replace('!', '').split() if not wordnet.synsets(word.lower(), lang='spa')])

print('Numero de palabras no incluidas en el corpus:', len(non_words))

### Preprocesamiento

Tras el análisis de los datos, realizaremos una serie de pasos de procesamiento para despues volver a analizar los datos y poder ver la evolución de estos. En primer lugar, para evitar el desbalance de las clases del dataset realizaremos un rebalanceo de los datos mediante undersampling. Con ello ayudaremos al modelo a no sobreentrenarse con aquella clase que sea mayoritaria en los datos de entrenamiento. 

In [ ]:
df_undersampled = undersample(df, 'RawContent', 'Sentiment')

Tras ello, aplicaremos una normalizacion al texto de los tweets con los siguientes pasos:
- Transformacion de todas las letras a minúsculas, para que el modelo no diferencie entre palabras por este factor, ya que el significado no varía
- Eliminacion de stopwords y tildes: las stopwords son palabras que no dan significado habitualmente a las oraciones, y quitar las tildes homogeniza aquellas palabras correctamente escritas e incorrectamente escritas (al igual que anteriormente, no queremos que el modelo diferencie entre palabras bien y mal escritas por tildes, aunque en este caso pueda haber excepciones que cambien su significado)
- Eliminacion de patrones de twitter: retirar user tags (@xxxx) y urls en tweets, que no aportan informacion a priori, también mejorará el rendimiento del modelo.
- Lemming: En muchas ocasiones el significado o relevancia de las palabras reside en su raiz, es decir, palabras como corredor, corriendo y corrí no aportan un significado de sentimiento distinto, por lo que el lemming, que homogeniza todas estas palabras (mantiene únicamente la raiz de la palabra) facilita la tarea del modelo. Se probó tambien el stemming, pero con las faltas de ortografía, el lemming tuvo un mejor rendimiento.

In [ ]:
df_undersampled['RawContent'] = df_undersampled['RawContent'].apply(lambda x: preprocess_text(x))

Una vez hecho el preprocessamiento, volvamos a hacer un breve analisis exploratorio para ver como han evolucionado los valores. En primer lugar, el rebalanceo de clases. Podemos ver como ahora disponemos del mismo numero de tweets de cada clase.

In [ ]:
df_undersampled['Sentiment'].value_counts().plot(kind='bar')
# Añadimos titulo y ylabel
plt.title('Número de tweets por sentimiento')
plt.ylabel('Número de tweets')

Pese a que la diferencia entre el numero de palabras por clases no ha variado demasiado en terminos relativos (sigue habiendo una diferencia de cerca de un 25%), la desviacion tipica de cada grupo si ha disminuido, lo cual puede indicar que hemos reducido mucho ruido gracias al preprocesamiento.

In [ ]:
# Distirbucion del numero de palabras por tweet por sentimiento
df_undersampled['Number of words'] = df_undersampled['RawContent'].apply(lambda x: len(x.split()))
df_undersampled.groupby('Sentiment')['Number of words'].plot(kind='kde', legend=True)


# Cuartiles del numero de palabras por tweet por sentimiento
df_undersampled.groupby('Sentiment')['Number of words'].describe()

No hemos retirado los emojis, por lo que las métricas serán las mismas, pero si podemos ver si el numero de palabras que no aparecen en el corpus de español de nltk ha disminuido. Podemos ver como más de la mitad de las palabras en principio problematicas, ya no lo son.

In [ ]:
non_words = set()
for tweet in df_undersampled['RawContent']:
    non_words.update([word for word in tweet.split() if not wordnet.synsets(word.lower(), lang='spa')])

print('Numero de palabras no incluidas en el corpus:', len(non_words))

In [ ]:
# Ejemplos de palabras no incluidas en el corpus (algunas deberian estarlo, pero el corpus no es perfecto)
list(non_words)[:50]

## Preparacion de los datos para los modelos

Una vez los datos se han preprocesado, se pepararán dos csvs con los datos listos para entrenar los modelos. El primero para los modelos RNN y LSTM, y el segundo para el modelo ROBERTA.

In [ ]:
# Datos para bert (sin tokenizar)
df_undersampled[['RawContent', 'Sentiment']].to_csv('preprocessed_tweets_bert.csv')

# Datos para lstm y rnn (tokenizados)
df_undersampled['RawContent'] = df_undersampled['RawContent'].apply(lambda x: preprocess_text(x, ['split_in_tokens']))
df_undersampled[['RawContent', 'Sentiment']].to_csv('preprocessed_tweets_lstm_rnn.csv')